In [ ]:
import chromadb
import chromadb.utils.embedding_functions as embedding_functions
import pandas as pd
from google import genai
from google.genai import types

# import
model = "gemini-2.5-flash-lite-preview-06-17"
api_key = "your_api_key_here"

# use directly
google_ef  = embedding_functions.GoogleGenerativeAiEmbeddingFunction(api_key=api_key, task_type="RETRIEVAL_QUERY", model_name="models/text-embedding-004")
client_db = chromadb.HttpClient(host='localhost', port=8000)
client_ai = genai.Client(api_key=api_key)


In [ ]:
result = client_ai.models.embed_content(
        model="models/text-embedding-004",
        contents="What is the meaning of life?",
        config=types.EmbedContentConfig(task_type="RETRIEVAL_DOCUMENT")
)

In [19]:
bible_df = pd.read_csv('data/t_web.csv')
bible_keys = pd.read_csv('data/key_english.csv')
bible_df = pd.merge(bible_df, bible_keys[['b','n']], on='b')
book_group = pd.pivot_table(bible_df, index=['n','c'], values='t', 
                            aggfunc=lambda x: ' '.join(x)).reset_index()

In [20]:
book_group["t"][0]

"Adam, Seth, Enosh, Kenan, Mahalalel, Jared, Enoch, Methuselah, Lamech, Noah, Shem, Ham, and Japheth. The sons of Japheth: Gomer, and Magog, and Madai, and Javan, and Tubal, and Meshech, and Tiras. The sons of Gomer: Ashkenaz, and Diphath, and Togarmah. The sons of Javan: Elishah, and Tarshish, Kittim, and Rodanim. The sons of Ham: Cush, and Mizraim, Put, and Canaan. The sons of Cush: Seba, and Havilah, and Sabta, and Raama, and Sabteca. The sons of Raamah: Sheba, and Dedan. Cush became the father of Nimrod; he began to be a mighty one in the earth. Mizraim became the father of Ludim, and Anamim, and Lehabim, and Naphtuhim, and Pathrusim, and Casluhim (from whence came the Philistines), and Caphtorim. Canaan became the father of Sidon his firstborn, and Heth, and the Jebusite, and the Amorite, and the Girgashite, and the Hivite, and the Arkite, and the Sinite, and the Arvadite, and the Zemarite, and the Hamathite. The sons of Shem: Elam, and Asshur, and Arpachshad, and Lud, and Aram, a

In [13]:
from tqdm import tqdm
import time

book_group_summary = []

for v in tqdm(book_group.to_numpy()):
    response = client_ai.models.generate_content(
    model=model,
    contents=f"""Summarize below bible chapter. 
    Detailed information like numbers, names must be accurately preserved.
    <CHAPTER>
    {v[2]}
    </CHAPTER>
    """)
    
    if not response.candidates:
        print("No candidates found for:", v[0], v[1])
        continue

    rc = response.candidates[0]
    
    if not rc.content or not rc.content.parts:
        print("No content or parts found for:", v[0], v[1])
        continue

    try:
        summary = rc.content.parts[0].text
    except:
        summary = "error"
        
    book_group_summary.append((v[0], v[1], v[2], summary))
    
    # there is limit for gemini-2.0-flash
    # 15 requests per minute
    # 1500 requests per day
    time.sleep(0.5) 



 39%|███▉      | 465/1189 [17:01<20:48,  1.72s/it] 

No content or parts found for: Genesis 17


100%|██████████| 1189/1189 [40:54<00:00,  2.06s/it]


In [49]:
book_summary_df = pd.DataFrame(book_group_summary, columns=['book', 'chapter', 'text', 'summary'
])
book_summary_df.to_csv('data/bible_summary.csv', index=False)
book_summary_df

,book,chapter,text,summary
0,1 Chronicles,1,"Adam, Sheth, Enosh, Kenan, Mahalaleel, Jered, ...","This chapter provides a detailed genealogy, be..."
1,1 Chronicles,2,"These are the sons of Israel; Reuben, Simeon, ...",This chapter provides a detailed genealogy of ...
2,1 Chronicles,3,"Now these were the sons of David, who were bor...",This chapter lists the sons of King David and ...
3,1 Chronicles,4,"The sons of Judah; Pharez, Hezron, and Carmi, ...",This chapter details the genealogies of the tr...
4,1 Chronicles,5,Now the sons of Reuben the first-born of Israe...,This chapter of the Bible begins by establishi...
...,...,...,...,...
1183,Zechariah,13,In that day there shall be a fountain opened t...,This chapter describes a future day of cleansi...
1184,Zechariah,14,"Behold, the day of the LORD cometh, and thy sp...","This chapter describes the Day of the LORD, wh..."
1185,Zephaniah,1,The word of the LORD which came to Zephaniah t...,The first chapter of Zephaniah begins with the...
1186,Zephaniah,2,"Assemble yourselves, yes, assemble, O nation n...","This chapter of the Bible, attributed to Zepha..."


In [50]:
client_db.delete_collection(name="bible_wbt")
collection = client_db.create_collection(name="bible_wbt", embedding_function=google_ef)

In [51]:
batch_size = 100
documents_batch = []
ids_batch = []
metadatas_batch = []

current_index = 0
for idx, v in tqdm(enumerate(bible_df.to_numpy()), total=bible_df.shape[0]):
    book = v[-1]
    chapter = v[2]
    verse = v[3]
    text = v[4]
    
    # Get the current verse index within the chapter
    chapter_verses = bible_df[(bible_df['n'] == book) & (bible_df['c'] == chapter)].reset_index(drop=True)
    current_verse_idx = chapter_verses[chapter_verses['v'] == verse].index[0]
    
    # Get previous and next verse text
    previous_verse = chapter_verses.iloc[current_verse_idx - 1]['t'] if current_verse_idx > 0 else None
    next_verse = chapter_verses.iloc[current_verse_idx + 1]['t'] if current_verse_idx < len(chapter_verses) - 1 else None
    
    summary = book_summary_df[(book_summary_df['book'] == book) & 
                                           (book_summary_df['chapter'] == chapter)]
    
    chapter_summary = summary['summary'].values[0] if not summary.empty else "No summary available"
    
    metadata = {
        'book': book,
        'chapter': chapter,
        'verse': verse,
        'text': text,
        'previous_verse': f"{book} {chapter}:{verse-1} - {previous_verse}" if previous_verse else 'None',
        'next_verse': f"{book} {chapter}:{verse+1} - {next_verse}" if next_verse else 'None',
        'chapter_summary': chapter_summary
    }
    
    # Add to batch
    documents_batch.append(text)
    ids_batch.append(f"{v[0]}")
    metadatas_batch.append(metadata)
    
    current_index = idx
    
    # When batch is full or it's the last item, add to collection
    if len(documents_batch) == batch_size or idx == len(bible_df) - 1:
        result = client_ai.models.embed_content(
            model="models/text-embedding-004",
            contents=documents_batch,
            config=types.EmbedContentConfig(task_type="RETRIEVAL_DOCUMENT")
        )
        
        if not result.embeddings:
            print("No embeddings returned for the batch.")
            continue
    
        valid_embeddings = []
        valid_documents = []
        valid_ids = []
        valid_metadatas = []

        for i, embedding in enumerate(result.embeddings):
            if embedding.values is not None:
                valid_embeddings.append(embedding.values)
                valid_documents.append(documents_batch[i])
                valid_ids.append(ids_batch[i])
                valid_metadatas.append(metadatas_batch[i])
            else:
                print(f"Warning: Embedding for document {ids_batch[i]} is None, skipping")
    
        if valid_embeddings:  # Only add if we have valid embeddings
            collection.add(
                documents=valid_documents,
                ids=valid_ids,
                metadatas=valid_metadatas,
                embeddings=valid_embeddings
        )
        
        print(f"Added batch of {len(documents_batch)} documents")
        
        # Clear batches for next iteration
        documents_batch = []
        ids_batch = []
        metadatas_batch = []


  1%|          | 182/31102 [00:02<07:08, 72.22it/s]

Added batch of 100 documents


  1%|          | 276/31102 [00:04<08:19, 61.77it/s]

Added batch of 100 documents


  1%|          | 369/31102 [00:06<07:23, 69.31it/s]

...

Added batch of 100 documents


100%|█████████▉| 31100/31102 [07:39<00:00, 57.46it/s]

Added batch of 100 documents


100%|██████████| 31102/31102 [07:39<00:00, 67.63it/s]

Added batch of 2 documents


In [60]:
collection = client_db.get_collection(name="bible_wbt", embedding_function=google_ef)
collection.query(
    query_texts=["Who made the world?"],
    n_results=5
)

{'ids': [['9031012', '9031009', '13010009', '9023027', '9017038']],
 'distances': [[1.8080245, 1.8217556, 1.8232312, 1.8242537, 1.8303963]],
 'embeddings': None,
 'metadatas': [[{'chapter_summary': "This chapter describes the defeat of the Israelites by the Philistines in a battle on Mount Gilboa. King Saul's sons, Jonathan, Abinadab, and Malchi-shua, were killed. Saul himself was severely wounded by archers. He asked his armor-bearer to kill him to prevent capture and abuse by the Philistines, but the armor-bearer refused out of fear. Saul then took his own sword and fell upon it, killing himself. His armor-bearer followed suit and died with him. Saul and his three sons died that same day.\n\nThe Philistines stripped the slain and found Saul and his sons on Mount Gilboa. They cut off Saul's head, stripped off his armor, and sent these trophies to their land to be displayed in their temples and among their people. Saul's armor was placed in the house of Ashtaroth, and his body was fast